In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 10.9MB/s 
     |████████████████████████████████| 2.9MB 53.4MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 
     |████████████████████████████████| 890kB 46.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1414ff901f69969474ec6b8c2d7026de2ed508f9fb153d5e7b218cc96478e5a0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import time
import datetime
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import json

import nltk
nltk.download('punkt')

import torch
import torch.nn.functional as F
import transformers as optimus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer_class, pretrained_weights = (
    optimus.BertTokenizer,
    "bert-base-uncased",
)

# load the tokenizer from pre-trained model
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, do_lower_case=True)

In [6]:
model_class, pretrained_weights = (
    optimus.BertForMultipleChoice,
    "bert-base-uncased",
)

# loading pre-trained bert model
model = model_class.from_pretrained(pretrained_weights, return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [17]:
def translateCSV(input_path, output_path):
    data = []
    with open(input_path) as f:
        for line in f:
            data.append(json.loads(line))

    df = pd.DataFrame(data)
    translatedCSV = { "premise":[], "choice1":[], "choice2": [], "question": [], "label": []}

    for index, row in df.iterrows():
        translatedCSV["premise"].append(row["premise"])
        translatedCSV["choice1"].append(row["choice1"])
        translatedCSV["choice2"].append(row["choice2"])
        translatedCSV["question"].append(row["question"])
        translatedCSV["label"].append(row["label"])

        if index%100 == 0:
            print(f'Finished {index} rows!!')

    df = pd.DataFrame(data=translatedCSV)
    df.to_csv(output_path, index=False)
    
translateCSV('./val.jsonl', './en-val.csv')

Finished 0 rows!!


In [38]:
val_data = pd.read_csv('./translated-gu-val.csv')
val_data = pd.read_csv('./translated-hi-val.csv')
val_data = pd.read_csv('./translated-mr-val.csv')

val_data = pd.read_csv('./en-val.csv')
val_data

,premise,choice1,choice2,question,label
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,1
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,0
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,1
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,1
...,...,...,...,...,...
95,My ears were ringing.,I went to a museum.,I went to a concert.,cause,1
96,I tidied up my house.,I was swamped with work.,I was expecting company.,cause,1
97,The airline mishandled my luggage.,They offered me compensation.,They cancelled my flight.,effect,0
98,The computer was expensive to fix.,I got it repaired.,I bought a new one.,effect,1


In [39]:
max_seq_len = 256
batch_size = 100

In [40]:
# initialize empty dataframe
val = pd.DataFrame()

# iterate each row in previously loaded train_data
# tokenizer(sent_a, sent_b) returns a dictionary of encoded values namely input_ids, attention_mask, token_type_ids
# input_ids - IDs returned by bert-tokenizer
# attention_mask - 1's for all the ids in the sentence and 0's for all padded tokens
# token_type_ids - 0's for all tokens in sent_a, 1's for all tokens in sent_b
for i, row in val_data.iterrows():
  d = {}
  q = row['question']
  encoded = tokenizer(
      [
       [row['premise']+f' What is the {q} of this?', row['premise']+f' What is the {q} of this?'],
       [row['choice1'], row['choice2']]
      ]
  )
  d['input_ids'] = encoded['input_ids']
  d['attention_mask'] = encoded['attention_mask']
  d['token_type_ids'] = encoded['token_type_ids']
  d['label'] = row['label']
  val = val.append(d, ignore_index=True)
  
# # padding all lists to MAX_SEQ_LENGTH
val['input_ids'] = val['input_ids'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])
val['attention_mask'] = val['attention_mask'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])
val['token_type_ids'] = val['token_type_ids'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])

In [41]:
val

,attention_mask,input_ids,label,token_type_ids
0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1996, 2158, 2357, 2006, 1996, 6904, 187...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1996, 2611, 2179, 1037, 11829, 1999, 20...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1996, 2450, 3394, 1012, 2054, 2003, 199...",0.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,..."
3,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1045, 2359, 2000, 27749, 2943, 1012, 20...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1996, 24575, 6240, 15005, 2094, 1012, 2...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...
95,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 2026, 5551, 2020, 13060, 1012, 2054, 20...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,..."
96,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1045, 14841, 10265, 2094, 2039, 2026, 2...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
97,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1996, 8582, 28616, 11774, 3709, 2026, 1...",0.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
98,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[101, 1996, 3274, 2001, 6450, 2000, 8081, 101...",1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [42]:
# convert all lists to pytorch tensors
input_ids = torch.tensor([r for r in val['input_ids']], dtype=torch.long)
attention_mask = torch.tensor([r for r in val['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([r for r in val['token_type_ids']], dtype=torch.long)
label_ids = torch.tensor([r for r in val['label']], dtype=torch.long)

In [43]:
# wraping tensors into TensorDataset
# Initialing DataLoader for efficiently loading data
val_dataset = TensorDataset(input_ids, attention_mask, token_type_ids, label_ids)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [44]:
# convert model to eval mode
model.to(device)
model.eval()

# initialize placeholders
total_eval_accuracy = 0
total_eval_loss = 0

# iterate for all batched in the dataloader
for step, batch in enumerate(tqdm(val_dataloader)):
    # unpack values in the batch
    input_ids, input_mask, segment_ids, label_ids = batch

    # load tensors to device
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    
    # perform forward pass without tracking gradients
    with torch.no_grad():        
        outputs = model(input_ids=input_ids,
                    attention_mask=input_mask,
                    token_type_ids=segment_ids,
                    labels=label_ids)

    # Computing softmax for finding max probablity
    logits = F.softmax(outputs['logits'], dim=-1)
    # move ground truths and predictions to cpu
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    # get index of max probability
    outputs = np.argmax(logits, axis=1)

    # comapre ground truths and predictions
    total_eval_accuracy += np.sum(outputs == label_ids)

avg_val_accuracy = total_eval_accuracy / (len(val_dataloader) * val_dataloader.batch_size)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

avg_val_loss = total_eval_loss / (len(val_dataloader) * val_dataloader.batch_size)

100%|██████████| 1/1 [00:03<00:00,  3.22s/it]

  Accuracy: 0.55


In [45]:
outputs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
og = val_data['label'].to_numpy()

In [47]:
sum(outputs == og)

55